# Setup

### Imports

In [ ]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

Load the TensorBoard notebook extension

%load_ext tensorboard

### Constants

In [ ]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
path_to_file = 'shakespeare.txt'
num_ns = 4

# Generate Training Data
    A tuple (target, context, label) tensors constitutes one training example
_(In skip-gram negative sampling Word2Vec model)_

- Batch 1 positive `context_word` and `num_ns` negative context words into one tensor;
- This produces a set of positive skip-grams _(labelled as 1)_ & negative samples _(labelled as 0)_ for each target word;
- Notice that the target is of shape (1,) while the context and label are of shape (1 + `num_ns`,)

### Main function

In [ ]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  '''
    - Generate training examples for the Word2Vec model;
    - Iterates over each word from each sequence to collect positive and negative context words;
    - Length of target, contexts and labels should be same, representing the total number of training examples;
    - Generates skip-gram pairs with negative sampling for a list of sequences (int-encoded sentences)
    based on window size, number of negative samples and vocabulary size.
  '''

  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens (word frequency rank).
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)

      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels


## Prepare training data

### Import text dataset

In [ ]:
# path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

with open(path_to_file) as f: 
  lines = f.read().splitlines()

# for line in lines[:20]:
#   print(line)

text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### Vectorize sentences

In [ ]:
# Now, create a custom standardization function to lowercase the text and remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), '')

# Define the vocabulary size and number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the TextVectorization layer to normalize, split, and map strings to
# integers. Set output_sequence_length length to pad all samples to same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Call adapt on the text dataset to create vocabulary.
vectorize_layer.adapt(text_ds.batch(1024))

# Save the created vocabulary for reference:
# Make a list of all vocabulary tokens sorted (descending) by their frequency
inverse_vocab = vectorize_layer.get_vocabulary()
# print(inverse_vocab[:20])

# Vectorize the data in text_ds: Generate vectors for each element in the text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### Obtain sequences for Datasets

- You now have a `tf.data.Dataset` of integer encoded sentences;
- To prepare DS for training a Word2Vec model, flatten the dataset into a list of sentence vector sequences;
- This is required as you would iterate over each sentence in the DS to produce positive & negative examples;

In [ ]:
sequences = list(text_vector_ds.as_numpy_iterator())
# print(len(sequences))

# for seq in sequences[:5]:
#   print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

### Generate training examples from sequences

In [45]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=num_ns,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

# print('\n')
# print(f"targets.shape: {targets.shape}")
# print(f"contexts.shape: {contexts.shape}")
# print(f"labels.shape: {labels.shape}")

100%|██████████| 32777/32777 [00:09<00:00, 3431.34it/s]


### Configure the dataset for performance

To perform efficient batching for the potentially large number of training examples, use the tf.data.Dataset API.

After this step, you would have a tf.data.Dataset object of (target_word, context_word), (label) elements to train your Word2Vec model!

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
# print(dataset)

# Add cache() and prefetch() to improve performance.
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

# Model & Training


- The Word2Vec model can be implemented as a classifier;
- It distinguishes between true context words _(from skip-grams)_ and false context words _(negative sampling)_;
- You can perform a dot product between the embeddings of target and context words to:
    - Obtain predictions for labels and;
    - To compute loss against true labels in the dataset;

## Subclassed Word2Vec Model


Use the Keras Subclassing API to define your Word2Vec model with the following layers.

With the subclassed model, you can define the `call()` function that accepts _(target, context)_ pairs which can then be passed into their corresponding embedding layer.

### Detailed explanation

Reshape the `context_embedding` to perform a dot product with `target_embedding` and return the flattened result.

__target_embedding__
- tf.keras.layers.Embedding layer which looks up the embedding of a word when it appears as a _target_ word;
- Number of parameters: (vocab_size * embedding_dim);

__context_embedding__
- tf.keras.layers.Embedding layer which looks up the embedding of a word when it appears as a _context_ word;
- Number of parameters: The same as those in `target_embedding`;

__dots__
- tf.keras.layers.Dot layer that computes the dot product of target and context embeddings from a training pair;

__flatten__
- tf.keras.layers.Flatten layer to flatten the results of dots layer into logits;

### Word2Vec Class

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      input_length=1,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim,
                                       input_length=num_ns+1)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots


## Define loss function and compile model
- You can use tf.keras.losses.CategoricalCrossEntropy as an alternative to the negative sampling loss;

### Custom Loss Function
- If you would like to write your own custom loss function, you can also do so as follows:

In [ ]:
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)

### Build the model
- Instantiate Word2Vec class with an embedding dimension of 128 _(you could experiment with different values)_;
- Compile the model with the tf.keras.optimizers.Adam optimizer;

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

# Also define a callback to log training statistics for tensorboard.
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

# Train the model with dataset prepared above for some number of epochs.
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

In [ ]:
%tensorboard --logdir logs

# Embedding Lookup and Analysis
Generate files to get the results

In [ ]:
# Obtain the weights from the model using get_layer() and get_weights()
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]

# The get_vocabulary() function provides the vocabulary to build a metadata file with one token per line
vocab = vectorize_layer.get_vocabulary()

# Create and save the vectors and metadata file. 
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

### Dowload result files

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass